In [5]:
from langchain.document_loaders import ReadTheDocsLoader,UnstructuredFileLoader


In [8]:
loader = UnstructuredFileLoader("./data/《我在亮剑当战狼》作者：寂寞剑客.txt")

In [9]:
docs = loader.load()


In [10]:
len(docs)


1

In [33]:
docs[0].metadata['source'].replace('./data/《我在亮剑当战狼》作者：寂寞剑客.txt', '我在亮剑当战狼》作者：寂寞剑客')


'我在亮剑当战狼》作者：寂寞剑客'

In [31]:
print(docs[0].metadata)

{'source': './data/《我在亮剑当战狼》作者：寂寞剑客.txt'}


In [44]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [66]:
tiktoken.encoding_for_model('gpt-3.5-turbo')


<Encoding 'cl100k_base'>

In [67]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)

In [68]:
chunks = text_splitter.split_text(docs[0].page_content)
len(chunks)

10659

In [69]:
tiktoken_len(chunks[0]), tiktoken_len(chunks[1])


(383, 375)

In [70]:
import hashlib
m = hashlib.md5()

In [87]:
from tqdm.auto import tqdm

documents = []

for doc in tqdm(docs):
    info = doc.metadata['source'].replace('./data/《我在亮剑当战狼》作者：寂寞剑客.txt', '我在亮剑当战狼》作者：寂寞剑客')
    m.update(info.encode('utf-8'))
    uid = m.hexdigest()[:12]
    chunks = text_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(chunks):
        documents.append({
            'id': f'{uid}-{i}',
            'text': chunk,
            'metadata': {'info': info}
        })

len(documents)

  0%|          | 0/1 [00:00<?, ?it/s]

10659

In [135]:
from tqdm.auto import tqdm
import requests
from requests.adapters import HTTPAdapter, Retry

batch_size = 100
endpoint_url = "http://localhost:8000"
s = requests.Session()

# we setup a retry strategy to retry on 5xx errors
retries = Retry(
    total=5,  # number of retries before raising error
    backoff_factor=0.1,
    status_forcelist=[500, 502, 503, 504]
)
s.mount('http://', HTTPAdapter(max_retries=retries))

for i in tqdm(range(0, 1000, batch_size)):
    i_end = min(len(documents), i+batch_size)
    # make post request that allows up to 5 retries
    res = s.post(
        f"{endpoint_url}/upsert",
        json={
            "documents": documents[i:i_end]
        }
    )

  0%|          | 0/10 [00:00<?, ?it/s]

In [136]:
queries = [
    {'query': "你知道我在亮剑当战狼这本书吗?"},

]

res = requests.post(
    f"{endpoint_url}/query",
    json={
        'queries': queries
    }
)
res

<Response [200]>

In [137]:
for query_result in res.json()['results']:
    query = query_result['query']
    answers = []
    scores = []
    for result in query_result['results']:
        answers.append(result['text'])
        scores.append(round(result['score'], 2))
    print("-"*70+"\n"+query+"\n\n"+"\n".join([f"{s}: {a}" for a, s in zip(answers, scores)])+"\n"+"-"*70+"\n\n")

----------------------------------------------------------------------
你知道我在亮剑当战狼这本书吗?

0.16: 围着火堆而坐的战狼便纷纷开始咒骂起魏大勇。  王喜奎一边骂一边捡起马肉干，掏出刺刀分切。  王野却走到那个鬼子战俘近前，抓起他的帆布挎包倒过来一抖，便从里边倒出来不少物件，还有一个日本记，居然还是带有徽章的笔记本。  早稻田大学留念？居然还是个大学生？  打开日记本一看，居然写的中文。  王野便心
0.17: �能啊。”十几头战狼愣了几秒钟，还是魏大勇最先反应过来，笑着说道，“就凭大王你的身手，就只有你杀鬼子的份，鬼子哪杀得了你？”  “屁，说谎你都不会说。”王野骂了一句，又说道，“都跟我走。”  “哎。”十几头战狼答应一声，纷纷翻身上马，就没有一个人问王野要去哪？
0.17: �天你跟俺一块跟警卫连打过架，就可以对俺呼来喝去，真把俺惹急了，揍死你个狗日的。  闷哼一声，魏大勇又道：“俺已经习惯了应有。”  听到这话，其他的队员便刷的看过来，脸上也流露出幸灾乐祸之色。  战狼小队是已经成立了，王野也已经被委任为小队长，但这绝不意味着这些队员就会对王野俯首贴耳、言听计从。
----------------------------------------------------------------------




In [138]:
res.json()['results']

[{'query': '你知道我在亮剑当战狼这本书吗?',
  'results': [{'id': '0e61cad1cb57-655',
    'text': '围着火堆而坐的战狼便纷纷开始咒骂起魏大勇。  王喜奎一边骂一边捡起马肉干，掏出刺刀分切。  王野却走到那个鬼子战俘近前，抓起他的帆布挎包倒过来一抖，便从里边倒出来不少物件，还有一个日本记，居然还是带有徽章的笔记本。  早稻田大学留念？居然还是个大学生？  打开日记本一看，居然写的中文。  王野便心',
    'metadata': {'source': 'chat',
     'source_id': None,
     'url': None,
     'created_at': None,
     'author': None,
     'document_id': '0e61cad1cb57-655'},
    'embedding': None,
    'score': 0.160749652449},
   {'id': '0e61cad1cb57-351',
    'text': '�能啊。”十几头战狼愣了几秒钟，还是魏大勇最先反应过来，笑着说道，“就凭大王你的身手，就只有你杀鬼子的份，鬼子哪杀得了你？”  “屁，说谎你都不会说。”王野骂了一句，又说道，“都跟我走。”  “哎。”十几头战狼答应一声，纷纷翻身上马，就没有一个人问王野要去哪？',
    'metadata': {'source': 'chat',
     'source_id': None,
     'url': None,
     'created_at': None,
     'author': None,
     'document_id': '0e61cad1cb57-351'},
    'embedding': None,
    'score': 0.165819300036},
   {'id': '0e61cad1cb57-152',
    'text': '�天你跟俺一块跟警卫连打过架，就可以对俺呼来喝去，真把俺惹急了，揍死你个狗日的。  闷哼一声，魏大勇又道：“俺已经习惯了应有。”  听到这话，其他的队员便刷的看过来，脸上也流露出幸灾乐祸之色。  战狼小队是已经成立了，王野